In [1]:
import pandas as pd

In [2]:
# Importacion de Plotly como libreria de DataViz
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [3]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


In [4]:
LastModelAcc = 0.7057186234817814

#### Leyendo datos:

In [5]:
# Generando el Dataframe apartir de los dataset RedTeam y BlueTeam
blueTeam = pd.read_csv( path_base + r"\data\procesed\blueTeam.csv")
blueTeam["Team"] = "Blue"
redTeam = pd.read_csv( path_base + r"\data\procesed\redTeam.csv")
redTeam["Team"] = "Red"
data = pd.concat([blueTeam,redTeam])

In [6]:
# Reemplazar los valores de la columna "Wins"
data['WinTags'] = data['Wins'].replace({0: 'Loss', 1: 'Win'})

# Crear un mapeo de colores
color_map = {'Win': 'yellow', 'Loss': 'gray'}
data.drop(columns=["Unnamed: 0"], inplace=True)

### Encoding:
- Para el proceso de encoding, fue necesario convertir la columna "Team" a valores numéricos, ya que contenía la declaración del color del equipo, ya sea "Red" o "Blue".

In [7]:
data = pd.get_dummies(data, columns=['Team'])

### Feature Engineering:
se han seleccionado los siguientes:
- Ratio de Kills/Deaths (puede indicar el rendimiento de un jugador).
- Ratio de WardsPlaced/WardsDestroyed (puede indicar la eficacia en el control del mapa).
- Ratio de GoldDiff/ExperienceDiff (puede reflejar el equilibrio entre los equipos).

In [8]:
features = ["WinTags","Kills", "Assists", "Deaths", "WardsPlaced", "WardsDestroyed", "GoldDiff", "ExperienceDiff"]

In [9]:
data = data[features]

### Separación de Datos:
- se dividio en un 80% de los datos para entrenamiento y un 20% para prueba.

In [10]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['WinTags'])  # X serán las características, excluyendo la columna objetivo (Wins)
y = data['WinTags']  # y será la variable objetivo (Wins)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Entrenamiento de un Modelo de Machine Learning:
- Se carga la libreria **sklearn** con el modelo **RandomForestClassifier** y se procede a realizar su entrenamiento

In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

### Evaluación del Modelo:
- Se evalúa el modelo con el conjunto de prueba utilizando métricas apropiadas.

In [12]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(report)


Accuracy: 0.7057186234817814
              precision    recall  f1-score   support

        Loss       0.69      0.72      0.71      1937
         Win       0.72      0.69      0.71      2015

    accuracy                           0.71      3952
   macro avg       0.71      0.71      0.71      3952
weighted avg       0.71      0.71      0.71      3952



### Ajuste y Optimización del Modelo:
- la declaracion de los Feature logra ajustar el modelo en un 70.57% de efectividad, una instancia mucho mas real que al correr con toda la tabla.

##### Ajuste de Hiperparámetros:
 - proceso de reentrenamiento para una mejor afinacion de resultado

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

##### Feature Importance:

- Utiliza la propiedad feature_importances_ de tu modelo para entender qué características son más importantes en la toma de decisiones del modelo. Puedes eliminar características menos importantes para simplificar el modelo.

In [14]:
feature_importances = best_model.feature_importances_

##### Validación Cruzada:

Realiza la validación cruzada para obtener una evaluación más robusta del rendimiento de tu modelo en diferentes conjuntos de datos.

In [15]:
from sklearn.model_selection import cross_val_score
import numpy as np

cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
average_cv_accuracy = np.mean(cv_scores)


##### Curva de Aprendizaje:

Plotea la curva de aprendizaje para entender si tu modelo se beneficiaría de más datos de entrenamiento o si está sobreajustado.

In [16]:
import plotly.graph_objects as go
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator, X, y, cv=5):
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv)

    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=train_sizes,
        y=train_scores_mean,
        mode='lines',
        name='Training Score',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=train_sizes,
        y=test_scores_mean,
        mode='lines',
        name='Cross-validation Score',
        line=dict(color='orange')
    ))

    fig.update_layout(
        title='Curva de Aprendizaje',
        xaxis_title='Tamaño del Conjunto de Entrenamiento',
        yaxis_title='Puntuación de Exactitud',
        legend=dict(x=0.02, y=0.95),
        margin=dict(l=0, r=0, t=30, b=0)
    )

    fig.show()
    fig.write_html(path_base + r"\\static\html\rf_prediccion.html")
    fig.write_image(path_base + r"\\static\img\rf_prediccion.png")

# Utiliza la función con tu mejor modelo y conjuntos de entrenamiento
plot_learning_curve(best_model, X_train, y_train)

#### Curva ROC

In [17]:
from sklearn.metrics import roc_curve, auc

# Obtener probabilidades de predicción para la clase positiva
y_probs = best_model.predict_proba(y_train)[:, 1]

c:\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



ValueError: could not convert string to float: 'Loss'

In [ ]:
# Calcular la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

In [ ]:
# Crear la figura interactiva de Plotly
fig = go.Figure()

# Agregar la curva ROC
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'Curva ROC (AUC = {roc_auc:.2f})'))

# Línea de referencia
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Línea de Referencia', line=dict(dash='dash')))

# Diseño del gráfico
fig.update_layout(
    title='Curva ROC con PCA y Random Forest',
    xaxis_title='Tasa de Falsos Positivos (FPR)',
    yaxis_title='Tasa de Verdaderos Positivos (TPR)',
    legend=dict(x=0.02, y=0.95),
    margin=dict(l=0, r=0, t=30, b=0)
)

# Mostrar la figura interactiva
fig.show()

##### Ensemble Methods:

Puedes probar métodos de conjunto (ensemble methods) como Bagging, Boosting, o Stacking para mejorar el rendimiento del modelo.

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagging_model = BaggingClassifier(base_estimator=best_model, n_estimators=10, random_state=42)
bagging_model.fit(X_train, y_train)

c:\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning:

`base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.



BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=50,
                                                        random_state=42),
                  random_state=42)

### Guardado del modelo

In [ ]:
if LastModelAcc < accuracy:
    import joblib
    joblib.dump(bagging_model, f'modelo_random_forest{str(round(accuracy,4))}.pkl')

['modelo_random_forest.pkl']

### Conclusión:
A pesar de que tanto el modelo base como el mejor modelo lograron una eficacia del 100%, estamos actualmente en la fase de evaluación de usabilidad para detectar posibles casos de sobreajuste. Se observa claramente una señal de sobreajuste debido a la presencia de registros anómalos, comúnmente conocidos como "Remontadas épicas". A pesar de la consistencia del equipo perdedor durante los primeros 30 minutos, hay instancias donde el equipo, inicialmente en desventaja, muestra una notable persistencia, revirtiendo el curso de la partida y siendo catalogado por el modelo como perdedor. Este fenómeno desafía la lógica del modelo al reconocer la victoria del equipo que, según su evaluación, debería haber perdido.

### Recursos
- [Use Random Forest to predict on Walmart Sales](https://medium.com/mlearning-ai/use-random-forest-to-predict-on-walmart-sales-ae6ebadb569b)
- [Isolation Forest](https://medium.com/@corymaklin/isolation-forest-799fceacdda4)
- [ML Dataset — How To Prepare a Good One?](https://betterprogramming.pub/ml-dataset-how-to-prepare-a-good-one-7d92ce1d45e5)
- [Use Random Forest to predict on a multiclass imbalance problem](https://medium.com/mlearning-ai/use-random-forest-to-predict-on-a-multiclass-imbalance-problem-fd0259098ac4)

In [ ]:
import subprocess
# Apagar
subprocess.run("shutdown -s")